In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
#langsmith tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loaded_data = WebBaseLoader("https://thecinemaholic.com/best-animal-documentaries-on-netflix/").load()
loaded_data

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://thecinemaholic.com/best-animal-documentaries-on-netflix/', 'title': '15 Best Animal Documentaries on Netflix Right Now', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n15 Best Animal Documentaries on Netflix Right Now\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\nSEARCH\n\n\n\n\n\n\n \n \n\n\n\nMovie News\nTV News\nUpcoming\nWhat to Watch\n\nNetflix\nHBO Max\nPrime Video\nHulu\nApple TV+\nDisney+\nPeacock\nParamount+\n\n\nMovie Explainers\nTV Explainers\nRecaps\nAbout Us\n  \n     \n\n     \n \n\n \n\n\n\n\n\n\n15 Best Animal Documentaries on Netflix (August 2025) \n Harsh Vardhan\nUpdated August 29, 2025\n\n\n\n\nFacebookTwitterPinterestRedditEmail \n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\nExploring the natural world through captivating storytelling and breathtaking cin

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splitted_data = splitter.split_documents(loaded_data)
splitted_data

[Document(metadata={'source': 'https://thecinemaholic.com/best-animal-documentaries-on-netflix/', 'title': '15 Best Animal Documentaries on Netflix Right Now', 'language': 'en-US'}, page_content='15 Best Animal Documentaries on Netflix Right Now\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\nSEARCH\n\n\n\n\n\n\n \n \n\n\n\nMovie News\nTV News\nUpcoming\nWhat to Watch\n\nNetflix\nHBO Max\nPrime Video\nHulu\nApple TV+\nDisney+\nPeacock\nParamount+\n\n\nMovie Explainers\nTV Explainers\nRecaps\nAbout Us\n  \n     \n\n     \n \n\n \n\n\n\n\n\n\n15 Best Animal Documentaries on Netflix (August 2025) \n Harsh Vardhan\nUpdated August 29, 2025\n\n\n\n\nFacebookTwitterPinterestRedditEmail'),
 Document(metadata={'source': 'https://thecinemaholic.com/best-animal-documentaries-on-netflix/', 'title': '15 Best Animal Documentaries on Netflix Rig

In [4]:
from langchain_openai import OpenAIEmbeddings

openai_embedding = OpenAIEmbeddings()


In [5]:
from langchain_community.vectorstores import FAISS

FAISS_data = FAISS.from_documents(splitted_data, openai_embedding)
FAISS_data

In [6]:
## Query From a vector db
query="wild babies"
result = FAISS_data.similarity_search(query)
result[0].page_content

'This cuddly docuseries features baby animals from across the world born in different environments and have to learn their respective ways of life. Narrated by Academy Award nominee Helena Bonham Carter, ‘Wild Babies’ has eight episodes, each of around 30 minutes, and features a baby fur seal named Tabo, who hails from Namibia, a female lion cub named Kaya, who hails from South Africa; we have Mina, a baby Orangutan from Indonesia, Quito, a baby mongoose and Silver, an Arctic fox, among others. You can stream ‘Wild Babies’ here.\n12. Life in Colour (2021)'

In [7]:
# So now lets use retriever chain and document chain.

# Create llm

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
# create document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Answer the following question, based only on the context given below:
    <context>
    {context}
    </context>
    
    question: {input}
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question, based only on the context guven below:\n    <context>\n    {context}\n    </context>\n    \n    question: {input}\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002221B4C5A20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002221B4C6A40>, root_client=<openai.OpenAI object at 0x000002221B4C4D90>, root_async_client=<openai.AsyncOpenAI object at 0x000002221B4C4BB0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=S

In [9]:
# create retrieval chain
from langchain.chains import create_retrieval_chain

FAISS_retriever = FAISS_data.as_retriever()

retrieval_chain = create_retrieval_chain(FAISS_retriever, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002221B4C4940>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question, based only on the context guven below:\n    <context>\n    {context}\n    </context>\n    \n    ques

In [10]:
response = retrieval_chain.invoke({"input":"wild babies"})
response['answer']

'"Wild Babies" is a docuseries featuring baby animals from different parts of the world as they learn to adapt to their environments. The show is narrated by Helena Bonham Carter and includes eight episodes, each about 30 minutes long, showcasing animals such as Tabo, a baby fur seal from Namibia, Kaya, a female lion cub from South Africa, Mina, a baby orangutan from Indonesia, Quito, a baby mongoose, and Silver, an Arctic fox.'

In [11]:
response

{'input': 'wild babies',
 'context': [Document(id='23ca45bb-f45a-4347-9c3d-a3dd26ff33c6', metadata={'source': 'https://thecinemaholic.com/best-animal-documentaries-on-netflix/', 'title': '15 Best Animal Documentaries on Netflix Right Now', 'language': 'en-US'}, page_content='This cuddly docuseries features baby animals from across the world born in different environments and have to learn their respective ways of life. Narrated by Academy Award nominee Helena Bonham Carter, ‘Wild Babies’ has eight episodes, each of around 30 minutes, and features a baby fur seal named Tabo, who hails from Namibia, a female lion cub named Kaya, who hails from South Africa; we have Mina, a baby Orangutan from Indonesia, Quito, a baby mongoose and Silver, an Arctic fox, among others. You can stream ‘Wild Babies’ here.\n12. Life in Colour (2021)'),
  Document(id='4d0a2fe2-19ce-496e-9841-a2cb2a9a9a41', metadata={'source': 'https://thecinemaholic.com/best-animal-documentaries-on-netflix/', 'title': '15 Best 

In [12]:
response['context']

[Document(id='23ca45bb-f45a-4347-9c3d-a3dd26ff33c6', metadata={'source': 'https://thecinemaholic.com/best-animal-documentaries-on-netflix/', 'title': '15 Best Animal Documentaries on Netflix Right Now', 'language': 'en-US'}, page_content='This cuddly docuseries features baby animals from across the world born in different environments and have to learn their respective ways of life. Narrated by Academy Award nominee Helena Bonham Carter, ‘Wild Babies’ has eight episodes, each of around 30 minutes, and features a baby fur seal named Tabo, who hails from Namibia, a female lion cub named Kaya, who hails from South Africa; we have Mina, a baby Orangutan from Indonesia, Quito, a baby mongoose and Silver, an Arctic fox, among others. You can stream ‘Wild Babies’ here.\n12. Life in Colour (2021)'),
 Document(id='4d0a2fe2-19ce-496e-9841-a2cb2a9a9a41', metadata={'source': 'https://thecinemaholic.com/best-animal-documentaries-on-netflix/', 'title': '15 Best Animal Documentaries on Netflix Right 